In [2]:
import tensorflow as tf
import numpy as np
import sklearn as sk
from sklearn.datasets import make_classification
import scikitplot as skplt
import pandas as pd
import copy

In [36]:
#############################
# PART I: Preparing Dataset.#
# @Elvis                    #
#############################
# The following code assumes that the 
# 1: .CSV files contains all hand gestures.
# 2: frame column exits. (This will be handled here).
# 3: 20 landmarks for each side of hand. (40 in total).
import os
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

# Macros
LANDMARKS_NUM            = 80
LANDMARS_OFFSET          = 1
INDEX_FRAME              = 1
INDEX_CAMERA_FACING_SIDE = 2

# Pre processing params
shuffle    = True         # Shuffle data before using.
dropout    = False        # Dropout regularization.
gradcheck  = False        # Gradient checking. 
moredata   = False        # Extend dataset to 
test_size  = 0.10         # Train, dev, test percentage. 

# Conditions
# conds = ["open_palm", "open_dorsal", "fist_palm", "fist_dorsal", "three_fingers_palm", "three_fingers_dorsal"]

# Loading all datasets (All .csv files) !TODO: Add script to check if correct imported.
paths = [os.path.join(r, f) for r,_,fs in os.walk('dataset') for f in fs if f.endswith('annotations.csv')]
dbase = [pd.read_table(path, sep=",", usecols = [n for n in range(LANDMARKS_NUM + 3) if n != INDEX_FRAME and n!= INDEX_CAMERA_FACING_SIDE]) for path in paths]
print(f'Row length should be 81 (80 landmarks + label). Got: {dbase[0].to_numpy().shape[1]}')

# Converting data into Numpy array
data = np.array([dbase[i].to_numpy() for i in range(len(dbase))]) # Dataset.
print(f'A total of {len(dbase)} datasets of shape {[d.shape for d in data]} datasets where added.')
data = np.concatenate(data) # Stack all datasets into one array.
print(f'The total concatenated dataset is: {data.shape}')

# Separting data and labels
X = data[:, LANDMARS_OFFSET:]
Y = data[:, 0]

# One hot encoding using Sklearn
label_encoder = LabelEncoder() # integer encode
integer_encoded = label_encoder.fit_transform(np.array(Y))
onehot_encoder = OneHotEncoder(sparse = False) # Binary encode
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
# print(onehot_encoded)

# Use the line below to invert element of index'IDX'.
#inverted = label_encoder.inverse_transform([np.argmax(onehot_encoded[IDX, :])])

# Split data into train and test sets. Shuffe data if param enabled.
X_train, X_test, Y_train, Y_test = train_test_split(X, onehot_encoded, test_size = test_size, shuffle=shuffle)
print(f'\nShape of training set\t: {X_train.shape}\nShape of test set\t: {X_test.shape}')

Row length should be 81 (80 landmarks + label). Got: 81
A total of 4 datasets of shape [(1323, 81), (1710, 81), (703, 81), (1246, 81)] datasets where added.
The total concatenated dataset is: (4982, 81)

Shape of training set	: (4483, 80)
Shape of test set	: (499, 80)


In [4]:
landmarks_0 = pd.read_table("dataset/Clement/ribes_clement_266_annotations.csv",sep=",")
landmarks_1 = copy.copy(landmarks_0)
del landmarks_1['frame']
del landmarks_1['camera_facing_side']
# landmarks_1

landmarks_2 = copy.copy(landmarks_1)
for i in range(len(landmarks_2)):
    label = landmarks_2.iloc[i]["source"]
    landmarks_2.at[i,'source'] = label[:-5]
# landmarks_2

landmarks_3 = copy.copy(landmarks_2)
classes = ["open_palm",
             "open_dorsal",
             "fist_palm",
             "fist_dorsal",
             "three_fingers_palm",
             "three_fingers_dorsal"]

for i in range(len(landmarks_3)):
    current_class = landmarks_3.at[i,'source']
    landmarks_3.at[i,'source'] = classes.index(current_class)
landmarks_3

landmarks_4 = landmarks_3.to_numpy(dtype=np.float32)
landmarks_4

landmarks_5 = np.copy(landmarks_4)
np.random.shuffle(landmarks_5)
landmarks_5[:700].shape

X_train, X_test, Y_train, Y_test = landmarks_5[:600,1:], landmarks_5[600:700,1:], landmarks_5[:600,0], landmarks_5[600:700,0]
encoder = sk.preprocessing.OneHotEncoder(dtype=np.float32)
encoder.fit(Y_train.reshape((-1,1)))

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float32'>,
              handle_unknown='error', sparse=True)

In [7]:
Y_test

array([2., 0., 0., 1., 0., 3., 4., 1., 4., 3., 2., 0., 3., 2., 1., 0., 1.,
       0., 2., 3., 5., 1., 2., 0., 1., 5., 2., 4., 3., 1., 1., 0., 1., 3.,
       2., 5., 4., 1., 5., 2., 2., 2., 1., 0., 0., 1., 0., 0., 0., 0., 1.,
       0., 0., 1., 3., 3., 2., 0., 4., 5., 3., 2., 1., 4., 1., 5., 2., 3.,
       1., 0., 4., 1., 5., 5., 4., 3., 3., 1., 5., 0., 2., 1., 0., 0., 3.,
       0., 1., 2., 3., 2., 3., 4., 3., 2., 2., 0., 4., 0., 0., 4.],
      dtype=float32)

In [6]:
Y_train_encoded=encoder.transform(Y_train.reshape((-1,1))).toarray()
Y_test_encoded=encoder.transform(Y_test.reshape((-1,1))).toarray()
print(Y_test_encoded[:5])

[[0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]


In [22]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train,Y_train_encoded))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test,Y_test_encoded))

ValueError: too many values to unpack (expected 2)

In [80]:
# Group lines in batches
BATCH_SIZE = 50
train_dataset_batched = train_dataset.batch(BATCH_SIZE)
test_dataset_batched = test_dataset.batch(BATCH_SIZE)

In [8]:
# this snippet will dissapear at some point in the future
# it exists because I found two bugs in KerasClassifier while writing
# this notebook. 
# Issues are raised in the tensorflow repo and this should be fixed soon(TM)

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier 

class KerasClassifier_Patched(KerasClassifier):
    # bugfix: classifier doesn't declare that it is a classifier
    # in the Scikit learn API
    _estimator_type = "classifier"
    
    # bugfix: the current wrapper does not work with HotOne encoded
    # labels
    # this is only a fix in the specific case of this notebook,
    # not a general one
    def score(self, x, y, **kwargs):
        _, accuracy = self.model.evaluate(x,y, verbose=0, **kwargs)
        return accuracy

In [24]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

def setupModel():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=(80)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(6))
    model.add(tf.keras.layers.Softmax())
    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-4),
                 loss=tf.keras.losses.CategoricalCrossentropy(),
                 metrics=[tf.keras.metrics.CategoricalAccuracy()])
    return model

mlp_model = KerasClassifier_Patched(build_fn=setupModel,
                               epochs=5,
                               batch_size=50,
                               verbose=1)

In [25]:
print(X_train.shape)
print(Y_train_encoded.shape)

(600, 80)
(600, 6)


In [26]:
mlp_model.fit(X_train,Y_train_encoded)

Train on 600 samples
Epoch 1/5
600/600 [==============================] - 0s 271us/sample - loss: 68.0428 - categorical_accuracy: 0.6783
Epoch 2/5
600/600 [==============================] - 0s 47us/sample - loss: 8.1237e-07 - categorical_accuracy: 1.0000
Epoch 3/5
600/600 [==============================] - 0s 55us/sample - loss: 8.0979e-07 - categorical_accuracy: 1.0000
Epoch 4/5
600/600 [==============================] - 0s 51us/sample - loss: 8.0701e-07 - categorical_accuracy: 1.0000
Epoch 5/5
600/600 [==============================] - 0s 31us/sample - loss: 8.0442e-07 - categorical_accuracy: 1.0000


In [27]:
test_accuracy = mlp_model.score(X_test, Y_test_encoded)

In [28]:
test_accuracy

1.0